<a href="https://colab.research.google.com/github/11310238-glitch/test01/blob/main/%E6%A0%A1%E5%9C%92%E4%BA%8C%E6%89%8B%E7%89%A9%E8%B3%87%E4%BA%A4%E6%8F%9B%E5%B9%B3%E8%87%BA%E5%8A%A9%E6%89%8B%E5%B7%A8%E9%87%8F_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#【版本一】MVP  
🛠️ 專案：校園二手物資交換平臺助手 (Campus Item Exchange CLI/GUI)

1.背景	大學生常常會有用不到的課本、文具、小型家電或家具等，直接丟棄很浪費。校內交換的需求雖然存在，但缺乏一個集中且易於查詢的資訊介面。

2.動機	建立一個以命令列介面 (CLI) 或輕量級圖形介面 (GUI) 運作的小型應用程式，讓使用者可以「刊登」想交換的物品，並「搜尋」別人刊登的物品。重點是資訊呈現與查詢功能。

3.目的	促進校園內物資的循環利用，幫助學生輕鬆找到或交換到需要的物品，達到節約與環保的目的。

In [2]:
# ==========================================
# 1. 安裝與設定
# ==========================================
# !pip install -q -U google-genai gradio pandas

import gradio as gr
import pandas as pd
import os
import uuid
from google import genai

# ⚠️ 請填寫你的 API Key
GOOGLE_API_KEY = "YOUR_API_KEY"

# 初始化 Client
AI_AVAILABLE = False
try:
    if GOOGLE_API_KEY == "YOUR_API_KEY" or not GOOGLE_API_KEY:
        print("⚠️ 警告：請填入有效的 API Key")
    else:
        client = genai.Client(api_key=GOOGLE_API_KEY)
        AI_AVAILABLE = True
        print("✅ AI 連線設定成功 (Gemini-2.5-Flash)")
except Exception as e:
    print(f"AI 設定失敗: {e}")

# ==========================================
# 2. 系統資料管理模組
# ==========================================
FILE_PATH = 'exchange_items.csv'
# 【修改 1】新增 '價格' 欄位
COLUMNS = ['ID', '名稱', '類別', '價格', '描述', '狀態', '聯絡資訊', '刊登日期']

def load_items():
    """從 CSV 檔案載入物品清單"""
    if os.path.exists(FILE_PATH):
        try:
            df = pd.read_csv(FILE_PATH)
            df['ID'] = df['ID'].astype(str)
            # 簡單防呆：如果舊檔案沒有價格欄位，補上空值
            if '價格' not in df.columns:
                df['價格'] = '未標示'
        except pd.errors.EmptyDataError:
            df = pd.DataFrame(columns=COLUMNS)
    else:
        df = pd.DataFrame(columns=COLUMNS)
    return df

def save_items(df):
    """將物品清單存回 CSV 檔案"""
    df.to_csv(FILE_PATH, index=False)

items_df = load_items()

# ==========================================
# 3. AI 功能模組
# ==========================================

def ai_generate_description(name, category, price):
    """AI 自動生成描述 (加入價格參考)"""
    if not AI_AVAILABLE: return "❌ AI 未連線，請檢查 API Key"
    if not name or not category: return "⚠️ 請先填寫「物品名稱」與「類別」"

    prompt = f"""
    你是一個校園二手拍賣的文案高手。
    請幫我為一個「{category}」類別的物品，名稱是「{name}」，
    價格設定為「{price}」，
    寫一段約 30-50 字的吸引人描述。
    語氣要活潑、適合大學生。
    請直接給出文案內容，不要有開場白。
    """
    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt
        )
        return response.text.strip()
    except Exception as e:
        return f"AI 生成失敗：{str(e)}"

def ai_shopping_assistant(user_query):
    """AI 導購員"""
    global items_df
    if not AI_AVAILABLE: return "❌ AI 未連線"

    active_items = items_df[items_df['狀態'] == '待交換']
    if active_items.empty: return "目前庫存是空的 😢"

    # 把價格也加入給 AI 的資訊中
    data_text = active_items[['ID', '名稱', '類別', '價格', '描述']].to_string(index=False)

    prompt = f"""
    你是校園二手交換平台的客服。以下是目前架上清單：
    {data_text}

    使用者問："{user_query}"

    請推薦符合的物品（列出名稱、價格和ID），若無則禮貌告知。
    """
    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt
        )
        return response.text.strip()
    except Exception as e:
        return f"AI 思考失敗：{str(e)}"

# ==========================================
# 4. 平台核心功能
# ==========================================

# 【修改 2】新增 price 參數
def add_item_to_list(name, category, price, description, contact):
    global items_df
    if not all([name, category, price, description, contact]):
        return "❌ 錯誤：所有欄位都必須填寫！", items_df

    new_id = str(uuid.uuid4())[:8]
    new_item = pd.DataFrame([{
        'ID': new_id,
        '名稱': name,
        '類別': category,
        '價格': price,  # 儲存價格
        '描述': description,
        '狀態': '待交換',
        '聯絡資訊': contact,
        '刊登日期': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')
    }])

    items_df = pd.concat([items_df, new_item], ignore_index=True)
    save_items(items_df)
    return f"✅ 成功刊登！物品ID: {new_id}。", items_df

def delete_item_by_id(item_id):
    global items_df
    if not item_id:
        return "❌ 錯誤：請輸入有效的物品 ID。", items_df
    initial_count = len(items_df)
    items_df = items_df[items_df['ID'] != item_id]
    if len(items_df) < initial_count:
        save_items(items_df)
        return f"✅ 成功刪除 ID 為 {item_id} 的物品。", items_df
    else:
        return f"❌ 刪除失敗：找不到 ID 為 {item_id} 的物品。", items_df

def search_and_filter(keyword, category_filter):
    df = items_df.copy()
    message = "✅ 這是目前的交換清單："
    if category_filter and category_filter != "所有類別":
        df = df[df['類別'] == category_filter]
        message = f"✅ 這是 '{category_filter}' 類別的物品清單："
    if keyword:
        df = df[df['名稱'].str.contains(keyword, case=False, na=False) |
                df['描述'].str.contains(keyword, case=False, na=False)]
        message = f"✅ 這是搜尋關鍵字 '{keyword}' 的結果清單："
    if df.empty:
        message = "❌ 找不到符合條件的物品。"
    return message, df

def update_item_status(item_id, new_status):
    global items_df
    if not item_id or not new_status:
        return "❌ 錯誤：請輸入 ID 並選擇新狀態。", items_df
    if item_id in items_df['ID'].values:
        items_df.loc[items_df['ID'] == item_id, '狀態'] = new_status
        save_items(items_df)
        return f"✅ 成功將 ID {item_id} 的狀態更新為 '{new_status}'。", items_df
    else:
        return f"❌ 狀態更新失敗：找不到 ID 為 {item_id} 的物品。", items_df

def get_categories():
    categories = items_df['類別'].unique().tolist()
    return ["所有類別"] + categories if categories else ["所有類別", "課本", "生活用品", "電子產品", "其他"]

# ==========================================
# 5. Gradio 介面
# ==========================================

with gr.Blocks(title="AI二手交易平台幫手") as demo:
    gr.Markdown("# 📚 校園二手物資交換平臺助手 (+AI 強化版)")
    gr.Markdown("歡迎使用校園交換助手！請在下方標籤頁切換功能。")

    status_message = gr.Textbox(label="系統訊息", interactive=False, value="系統啟動成功！")

    item_display = gr.Dataframe(
        value=items_df,
        headers=COLUMNS,
        label="物品交換清單",
        interactive=False,
        row_count=(1, "dynamic"),
        col_count=len(COLUMNS)
    )

    with gr.Tabs():
        # --- 分頁 1: 物品刊登 ---
        with gr.Tab("物品刊登 🚀"):
            gr.Markdown("### 刊登新物品")
            with gr.Row():
                with gr.Column(scale=1):
                    name_in = gr.Textbox(label="物品名稱", placeholder="例如：微積分課本 (上)")
                    category_in = gr.Dropdown(get_categories(), label="類別", value="課本", allow_custom_value=True)
                    # 【修改 3】新增價格輸入框
                    price_in = gr.Textbox(label="價格 / 交換條件", placeholder="例如：100元、請我喝飲料、免費")
                    contact_in = gr.Textbox(label="聯絡資訊", placeholder="Email/Line ID")

                with gr.Column(scale=1):
                    gr.Markdown("👇 **不知道怎麼寫？按按鈕讓 AI 幫你寫！**")
                    ai_desc_btn = gr.Button("✨ AI 自動生成描述", variant="secondary")
                    desc_in = gr.Textbox(label="簡短描述", placeholder="例如：九成新，書皮有輕微摺痕", lines=3)

            add_btn = gr.Button("刊登物品", variant="primary")

        # --- 分頁 2: 搜尋與篩選 ---
        with gr.Tab("搜尋與篩選 🔍"):
            gr.Markdown("### 搜尋交換物品")
            keyword_search = gr.Textbox(label="關鍵字搜尋", placeholder="輸入物品名稱或描述中的字詞")
            category_filter_dd = gr.Dropdown(get_categories(), label="依類別篩選", value="所有類別")
            search_btn = gr.Button("開始搜尋/顯示所有")

        # --- 分頁 3: AI 導購 ---
        with gr.Tab("🤖 AI 智能導購"):
            gr.Markdown("### 不知道要換什麼？問問 AI！")
            ai_query = gr.Textbox(label="你的需求", placeholder="例如：我想找 100 元以下的課本")
            ai_ask_btn = gr.Button("詢問 AI")
            ai_result = gr.Markdown(label="推薦結果")

        # --- 分頁 4: 狀態管理 ---
        with gr.Tab("狀態管理 (刊登者) 🔧"):
            gr.Markdown("### 物品狀態變更與刪除")
            status_id_in = gr.Textbox(label="物品 ID", placeholder="輸入要變更狀態的物品 ID")
            status_dd = gr.Dropdown(['待交換', '預訂', '已成交'], label="選擇新狀態", value='待交換')
            update_btn = gr.Button("更新物品狀態")

            gr.Markdown("---")
            delete_id_in = gr.Textbox(label="物品 ID", placeholder="輸入要刪除的物品 ID")
            delete_btn = gr.Button("刪除物品", variant="stop")

    # --- 事件處理 ---

    # 1. AI 生成描述 (把價格也傳進去)
    ai_desc_btn.click(ai_generate_description, inputs=[name_in, category_in, price_in], outputs=[desc_in])

    # 2. AI 導購
    ai_ask_btn.click(ai_shopping_assistant, inputs=[ai_query], outputs=[ai_result])

    # 3. 刊登 (加入 price_in)
    add_btn.click(add_item_to_list, inputs=[name_in, category_in, price_in, desc_in, contact_in], outputs=[status_message, item_display])

    # 4. 其他功能
    search_btn.click(search_and_filter, inputs=[keyword_search, category_filter_dd], outputs=[status_message, item_display])
    update_btn.click(update_item_status, inputs=[status_id_in, status_dd], outputs=[status_message, item_display])
    delete_btn.click(delete_item_by_id, inputs=[delete_id_in], outputs=[status_message, item_display])

demo.launch(share=True, debug=True)

⚠️ 警告：請填入有效的 API Key
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5bf695d66fb22d0646.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5bf695d66fb22d0646.gradio.live
